## Scraping Detik

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [3]:
jumlah_index = 50
threads_link = []
links = []
results = []
keywords = 'anies'

In [4]:
def scrape_links(page_number,keywords):
    url = f"https://www.detik.com/search/searchall?query={keywords}&sortby=time&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [5]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 9 links from page 6
Scraped 9 links from page 11
Scraped 9 links from page 9
Scraped 9 links from page 2
Scraped 9 links from page 12
Scraped 9 links from page 10
Scraped 9 links from page 18
Scraped 9 links from page 19
Scraped 9 links from page 27
Scraped 9 links from page 26
Scraped 9 links from page 7
Scraped 9 links from page 28
Scraped 9 links from page 13
Scraped 9 links from page 14
Scraped 9 links from page 34
Scraped 9 links from page 15
Scraped 9 links from page 4
Scraped 9 links from page 17
Scraped 9 links from page 1
Scraped 9 links from page 5
Scraped 9 links from page 20
Scraped 9 links from page 3
Scraped 9 links from page 37
Scraped 9 links from page 21
Scraped 9 links from page 30
Scraped 9 links from page 16
Scraped 9 links from page 38
Scraped 9 links from page 24
Scraped 9 links from page 8
Scraped 9 links from page 42
Scraped 9 links from page 49
Scraped 9 links from page 35
Scraped 9 links from page 40
Scraped 9 links from page 25
Scraped 9 links from pa

In [6]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "detail__title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "detail__author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "detail__date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('div',{"class": "page__breadcrumb"})
            if category_elements:
                category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail__body"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [7]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [8]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 450


,title,keywords,author,category,date,content,link
0,"Khofifah Dilirik Jadi Ketua Timses Prabowo, PK...",anies,Arief Ikhsanudin,Pemilu,"Senin, 25 Sep 2023 05:35 WIB",Gerindra Jawa Timur (Jatim) menyebut Gubernur ...,https://news.detik.com/pemilu/d-6948625/khofif...
1,"Survei Capres Indo Riset: Prabowo 34,8%, Ganja...",anies,Anggi Muliawati,Pemilu,"Senin, 25 Sep 2023 16:07 WIB",Lembaga Indo Riset merilis hasil survei terkai...,https://news.detik.com/pemilu/d-6949716/survei...
2,"Tanggapi Isu Pilpres 2 Poros, Cak Imin Berhara...",anies,Author not found,Category not found,"6,189 Views | Sabtu, 23 Sep 2023 15:01 WIB",Bakal Cawapres Koalisi Perubahan sekaligus Ket...,https://news.detik.com/detiktv/d-6946872/tangg...
3,Anies di Makassar: Perubahan Akan Makin Cepat ...,anies,Indra Komara,Pemilu,"Minggu, 24 Sep 2023 11:17 WIB",Pasangan bacapres dan bacawapres Koalisi Perub...,https://news.detik.com/pemilu/d-6947760/anies-...
4,"Bos PPI Nilai AMIN Bubar Jalan, Sahroni: Apa B...",anies,Matius Alfons Hutajulu,Pemilu,"Sabtu, 23 Sep 2023 20:44 WIB",Direktur Parameter Politik Indonesia Adi Prayi...,https://news.detik.com/pemilu/d-6947262/bos-pp...
5,"Urus SKCK, Anies Diantar 'Motor Sejuta Umat' H...",anies,Tim detikcom,Profil,"Senin, 25 Sep 2023 14:04 WIB",Bakal calon presiden Anies Baswedan mengurus S...,https://oto.detik.com/profil/d-6949416/urus-sk...
6,Bos PPI: Kemungkinan 2 Poros Ganjar Vs Prabowo...,anies,Author not found,Category not found,"77,271 Views | Sabtu, 23 Sep 2023 22:27 WIB",Direktur Eksekutif Parameter Politik Indonesia...,https://news.detik.com/detiktv/d-6947373/bos-p...
7,Cak Imin Ingatkan Parpol Harus Waspada Usai Ka...,anies,Amir Baihaqi,Berita,"Selasa, 26 Sep 2023 01:01 WIB",Bacawapres Koalisi Perubahan Muhaimin Iskandar...,https://www.detik.com/jatim/berita/d-6950545/c...
8,NasDem Siap Habis-habisan Menangkan Anies-Cak ...,anies,Dwiky Maulana Vellayati,Berita,"Selasa, 26 Sep 2023 01:45 WIB",Dewan Pimpinan Daerah (DPD) Partai NasDem Kabu...,https://www.detik.com/jabar/berita/d-6949637/n...
9,Cak Imin Yakin AMIN Menang Besar Pilpres di Jatim,anies,Deny Prasetyo Utomo,Berita,"Selasa, 26 Sep 2023 02:00 WIB",Bacawapres Koalisi Perubahan Muhaimin Iskandar...,https://www.detik.com/jatim/berita/d-6950556/c...


In [9]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 427


,title,keywords,author,category,date,content,link
0,"Khofifah Dilirik Jadi Ketua Timses Prabowo, PK...",anies,Arief Ikhsanudin,Pemilu,"Senin, 25 Sep 2023 05:35 WIB",Gerindra Jawa Timur (Jatim) menyebut Gubernur ...,https://news.detik.com/pemilu/d-6948625/khofif...
1,"Survei Capres Indo Riset: Prabowo 34,8%, Ganja...",anies,Anggi Muliawati,Pemilu,"Senin, 25 Sep 2023 16:07 WIB",Lembaga Indo Riset merilis hasil survei terkai...,https://news.detik.com/pemilu/d-6949716/survei...
3,Anies di Makassar: Perubahan Akan Makin Cepat ...,anies,Indra Komara,Pemilu,"Minggu, 24 Sep 2023 11:17 WIB",Pasangan bacapres dan bacawapres Koalisi Perub...,https://news.detik.com/pemilu/d-6947760/anies-...
4,"Bos PPI Nilai AMIN Bubar Jalan, Sahroni: Apa B...",anies,Matius Alfons Hutajulu,Pemilu,"Sabtu, 23 Sep 2023 20:44 WIB",Direktur Parameter Politik Indonesia Adi Prayi...,https://news.detik.com/pemilu/d-6947262/bos-pp...
5,"Urus SKCK, Anies Diantar 'Motor Sejuta Umat' H...",anies,Tim detikcom,Profil,"Senin, 25 Sep 2023 14:04 WIB",Bakal calon presiden Anies Baswedan mengurus S...,https://oto.detik.com/profil/d-6949416/urus-sk...
8,NasDem Siap Habis-habisan Menangkan Anies-Cak ...,anies,Dwiky Maulana Vellayati,Berita,"Selasa, 26 Sep 2023 01:45 WIB",Dewan Pimpinan Daerah (DPD) Partai NasDem Kabu...,https://www.detik.com/jabar/berita/d-6949637/n...
10,"Adu Kuat Tim Pemenangan Anies Vs Ganjar, Bagai...",anies,Author not found,Category not found,"13,495 Views | Sabtu, 23 Sep 2023 09:46 WIB",Pasangan Anies Baswedan-Muhaimin Iskandar (Cak...,https://news.detik.com/detiktv/d-6946464/adu-k...
11,"Makan Sate di Sukabumi, Anies Baswedan Terpika...",anies,Siti Fatimah,Berita Boga,"Kamis, 21 Sep 2023 11:30 WIB",Anies Baswedan melakukan kunjungan ke Kota dan...,https://food.detik.com/berita-boga/d-6942591/m...
12,Kata Cak Imin soal Peluang Khofifah Masuk Tim ...,anies,Deny Prasetyo Utomo,Berita,"Selasa, 26 Sep 2023 01:30 WIB",Bacawapres Koalisi Perubahan Muhaimin Iskandar...,https://www.detik.com/jatim/berita/d-6950551/k...
13,"Tanggapi Survei AMIN Buncit di Jatim, Cak Imin...",anies,Amir Baihaqi,Berita,"Selasa, 26 Sep 2023 02:30 WIB",Bacawapres Koalisi Perubahan Muhaimin Iskandar...,https://www.detik.com/jatim/berita/d-6950558/t...


Save Data

In [10]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/detik_{keywords}_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')